In [6]:
from glob import glob
from os import path
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerTuple
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pandas as pd
import os
import numpy as np
import re
from os.path import basename, splitext

# from solver import Instance

from argparse import Namespace
import sys
sys.path.append('../solver')
# from solver_output import practice_print
from solver_rostering import run_roster_solver_objval

In [7]:
list_city_dt = []

for file_in in os.listdir(r'../instances'):
    base_file = splitext(basename(file_in))[0]
    base_file_split = base_file.split('_')
    file = base_file_split[0]+'_'+base_file_split[1]
    list_city_dt.append(file)

list_city_dt = list(set(list_city_dt))
print(list_city_dt)
print(len(list_city_dt))

['berlin_db=2.00', 'frankfurt_db=0.50', 'frankfurt_db=1.00', 'berlin_db=0.50', 'frankfurt_db=2.00', 'paris_db=4.00', 'lyon_db=1.00', 'paris_db=1.00', 'berlin_db=1.00', 'frankfurt_db=4.00', 'lyon_db=0.50', 'berlin_db=4.00', 'lyon_db=4.00', 'paris_db=2.00', 'paris_db=0.50', 'lyon_db=2.00']
16


In [8]:
list_results = []

RM = 1.5
GM = 0.8
h_min = 32
h_max = 48
max_n_diff = 3

# for base_file in list_city_dt:
for base_file in ['lyon_db=0.50', 'lyon_db=1.00', 'lyon_db=2.00', 'lyon_db=4.00']:

    #create inputs to run a trial workforce size
    weekday_ = f'{base_file}_dt=doublepeak.json'
    weekend_ = f'{base_file}_dt=uniform.json'

    instance_file_weekday = f"../instances/{weekday_}"
    instance_file_weekend = f"../instances/{weekend_}"

    shift_file_weekday = f"../shifts/{weekday_}"
    shift_file_weekend = f"../shifts/{weekend_}"

    expand_workforce_to_regions=True
    workforce_dict = {}

    # for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:
    for OC in [1.2]:
        # for model in ['fixed', 'flex', 'partflex']:
        for model in ['fixed', 'flex', 'partflex']:
            if model == 'partflex':
                for max_n_shifts in range(2,5):
                    dict_results = run_roster_solver_objval(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff, max_n_shifts, expand_workforce_to_regions=True)
                    df_ = pd.DataFrame(dict_results)
                    list_results.append(df_)
            else:
                dict_results = run_roster_solver_objval(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff, expand_workforce_to_regions=True)
                df_ = pd.DataFrame(dict_results)
                list_results.append(df_)


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 37299 rows, 43328 columns and 466992 nonzeros
Model fingerprint: 0x8c09fe9d
Variable types: 26880 continuous, 16448 integer (16448 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 1.792000e+12
Presolve removed 34724 rows and 37649 columns
Presolve time: 1.00s
Presolved: 2575 rows, 5679 columns, 26385 nonzeros
Found heuristic solution: objective 1751.9066667
Variable types: 656 continuous, 5023 integer (5023 binary)

Root relaxation: objective 1.538993e+03, 6414 iterations, 0.30 seconds (0.35 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Tim

KeyboardInterrupt: 

In [ ]:
df_trials = pd.concat(list_results, ignore_index = True)
df_trials


,instance,city,demand_baseline,outsourcing_cost_multiplier,region_multiplier,global_multiplier,model,max_n_shifts,workforce_size,wage_costs,objective_value,objective_value_post_wage
0,lyon_db=0.50,lyon,0.5,1.2,1.5,0.8,fixed,NaN,69,1240.0,1538.993333,298.993333
1,lyon_db=1.00,lyon,1.0,1.2,1.5,0.8,fixed,NaN,112,2340.0,2709.802222,369.802222
2,lyon_db=2.00,lyon,2.0,1.2,1.5,0.8,fixed,NaN,195,4244.0,4933.397452,689.397452
3,lyon_db=4.00,lyon,4.0,1.2,1.5,0.8,fixed,NaN,377,8160.0,9444.638431,1284.638431
